## Library

In [ ]:
import urllib.request
import cv2
import numpy as np
import pytesseract
import os

## URL and set im

In [ ]:
url = 'http://192.168.3.56/cam-hi.jpg'
im = None

## Vòng While để lấy hình ảnh

In [ ]:
while True:
    cv2.namedWindow("Test", cv2.WINDOW_AUTOSIZE)
    
    img_resp = urllib.request.urlopen(url)
    imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
    image = cv2.imdecode(imgnp, -1)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Áp dụng lọc Gaussian để làm mượt hình ảnh
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')
    plates = plate_cascade.detectMultiScale(blurred, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    plate_images = []
    for (x, y, w, h) in plates: 
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        plate_img = gray[y:y + h, x:x + w]
        plate_images.append(plate_img)

    # Hiển thị ảnh gốc và ảnh xám
    cv2.imshow('Test Original', image)
    cv2.imshow('Test Grayscale', gray)

    # Nhận diện ký tự 
    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

## Đọc tên biển

In [ ]:
for plate_image in plate_images:
    plate_text = pytesseract.image_to_string(plate_image,config='--psm 8')
plate_text.strip()

## Xet xem bien co hop le hay khong

In [ ]:
# patterns = [r'^\d{2}[A-Z]{1}\s\d{3}\.\d{2}$',  # Pattern cho định dạng 1
#             r'^\d{2}[A-Z]{1}\s\d{2,3}\.\d{2}$']  # Pattern cho định dạng 2

## Luu anh ve file

In [ ]:
output_dir = 'recovery'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Lưu các hình ảnh vào thư mục recovery
for i, plate_img in enumerate(plate_images):
    filename = os.path.join(output_dir, f'plate_{i}.png')
    cv2.imwrite(filename, plate_img)
    print(f'Saved {filename}')